# Project 3: Getting Started 

This notebook is intended to help you get off to a flying start with the cars dataset. You don't have to use this notebook and you can discard any parts you do not like, they are purely intended as a help to get started. 

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

# optimization
from scipy import optimize

import statsmodels.formula.api as smf
import funcs 
import estimation

%load_ext autoreload
%autoreload 2

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [3]:
cars = pd.read_csv('cars.csv')
lbl_vars = pd.read_csv('labels_variables.csv')
lbl_vals = pd.read_csv('labels_values.csv')

# convert from dataframe to dict
lbl_vals = {c: lbl_vals[c].dropna().to_dict() for c in lbl_vals.columns}
lbl_vars.set_index('variable', inplace=True)

## Overview of the dataset

In [3]:
#lbl_vars.join(cars.mean(numeric_only=True).apply(lambda x: f'{x: .2f}').to_frame('Mean'))

# Set up for analysis

In [4]:
price_var = 'princ'
cars['logp'] = np.log(cars[price_var])

# new variable: price elasticity heterogeneous for home-region 
cars['logp_x_home'] = cars[price_var] * cars['home']

### Dummy variables

For working with matrices, we want to have a column for each dummy variable. 

In [5]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

### `x_vars`: List of regressors to be used 

In [6]:
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li'] + x_vars_dummies # <--- !!! choose your preferred variables here 
print(f'K = {len(x_vars)} variables selected.')

K = 38 variables selected.


In [7]:
K = len(x_vars) # number of x variables
N = cars.ma.nunique() * cars.ye.nunique() # 30 years, 5 markets = 150 obs
J = 40
x = cars[x_vars].values.reshape((N,J,K))
y = cars['s'].values.reshape((N,J))

In [8]:
# Rescale large values for cy, we

cy_i = x_vars.index('cy')
x[:,:,cy_i] = x[:,:,cy_i]/1_000

we_i = x_vars.index('we')
x[:,:,we_i] = x[:,:,we_i]/1_000

hp_i = x_vars.index('hp')
x[:,:,hp_i] = x[:,:,hp_i]/1_000

## Estimating the Multinomial Logit

In [9]:
theta0 = funcs.starting_values(x)

In [10]:
res = estimation.estimate(funcs.q,theta0,y,x, cov_type='Sandwich')

Optimization terminated successfully.
         Current function value: 0.086893
         Iterations: 227
         Function evaluations: 8931
         Gradient evaluations: 229
Computing numerical Hessian, expect approx. 12.17 seconds (for 779 criterion evaluations)


In [11]:
estimation.print_table(x_vars,res)

Optimizer succeded after 227 iter. (8931 func. evals.). Final criterion:  0.08689.
Results


theta      se        t
logp        -0.5145  0.1301  -3.9550
home         1.3744  0.0316  43.4428
cy          -0.3759  0.0825  -4.5583
hp          -0.8873  1.6293  -0.5446
we           0.4743  0.1792   2.6465
li          -0.0414  0.0172  -2.3996
MCC         -0.6402  0.4611  -1.3885
VW           0.3111  0.0477   6.5242
alfa romeo  -0.8453  0.0677 -12.4906
audi         0.0036  0.0135   0.2645
citroen     -0.4678  0.0418 -11.2018
daewoo      -0.2453  0.5034  -0.4872
daf         -0.5731  0.1237  -4.6329
fiat        -0.1021  0.0581  -1.7561
ford         0.1407  0.0541   2.6023
honda       -0.2314  0.0621  -3.7237
hyundai     -0.5578  0.1995  -2.7958
innocenti   -1.3802  0.1994  -6.9228
lancia      -0.8926  0.0629 -14.1931
mazda       -0.3484  0.0594  -5.8638
mercedes     0.5503  0.0712   7.7311
mitsubishi  -0.2810  0.0637  -4.4111
nissan      -0.1452  0.0531  -2.7355
opel         0.0814  0.0537   1.5151
peugeot      0.0077  0.0372   0.2072
renault      0.0726  0.0394   1.8408
rover       -0.3709  0.0833  -4.4520
saab        -0.3152  0.3211  -0.9816
seat        -0.6143  0.0658  -9.3314
skoda       -0.7738  0.1439  -5.3789
suzuki      -0.8779  0.1509  -5.8172
tal/hillman -1.3915  0.2000  -6.9584
tal/matra   -0.6825  0.5542  -1.2316
tal/simca   -0.3773  0.1093  -3.4532
tal/sunb    -0.6990  1.0235  -0.6830
talbot      -0.3732  0.0694  -5.3772
toyota      -0.0748  0.0556  -1.3451
volvo        0.0658  0.0895   0.7351

## Home Bias - Average Partial Effects

In [26]:
# Initiate thetahat and beta, the parameter for log price
thetahat = res['theta']
beta = thetahat[0] #parameter for log price

In [27]:
# Original choice probabilities
ccp_orig = funcs.choice_prob(thetahat,x)

In [28]:
# Calculate elasticities
elas = beta*(1-ccp_orig) # 1 partial effect for each car in each market

elas_home = np.mean(elas[x[:,:,1]==1]) # index = only those cars where home variable is equal to 1
elas_foreign = np.mean(elas[x[:,:,1]!=1]) # index = only those cars where home variable is equal to 0

print(f'Elasticity for "home" cars:    {elas_home.round(4)}')
print(f'Elasticity for "foreign" cars: {elas_foreign.round(4)}')

# standard errors on their way..

Elasticity for "home" cars:    -0.4921
Elasticity for "foreign" cars: -0.5061
